In [1]:
import sys
sys.path.append('..')
import os
from os.path import expanduser

import lcatools

In [2]:
from lcatools.tools import archive_from_json

In [3]:
elcd3_file = os.path.join(os.path.expanduser('~'),'Dropbox', 'data', 'ELCD', 'ELCD3.2.zip')
catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')
lcia_file = os.path.join(catalog_dir, 'elcd_lcia.json.gz')

In [4]:
def lcia(process, quantity):
    print(', '.join([quantity[desc] for desc in ('Method', 'Category', 'Indicator')]))
    print('-'*60)
    score = 0.0
    for x in process.exchanges():
        if x.flow.cf(lcia) != 0:
            c = x.flow.cf(quantity)
            v = x.value
            score += c*v
            print('%10.3g x %-10.3g = %-10.3g %s' % (c, v, c*v, x.flow['Name']) )
    print('='*60)
    print('             Total score: %g [%s]' % (score, lcia.reference_entity.unitstring()))
    return score

### ELCD LCIA Computation

In [4]:
l = archive_from_json(lcia_file)

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz:
Found Extension: zip
**Upstream reference encountered: /home/b/Dropbox/data/ELCD/ELCD3.2.zip

54 new quantity entities added (54 total)
1256 new flow entities added (1256 total)
0 new process entities added (0 total)


In [5]:
from lcatools.providers.ilcd import IlcdArchive

In [6]:
ps = IlcdArchive(elcd3_file, upstream=l)

Found Extension: zip


In [7]:
ps.retrieve_or_fetch_entity(ps.list_objects('Process')[27])

In [8]:
p = _

In [9]:
qs = (x for x in l.quantities() if 'Method' in x.keys())

In [10]:
q = next(qs)

In [11]:
q.keys()

dict_keys(['Comment', 'Method', 'Name', 'origin', 'Category', 'ReferenceYear', 'Duration', 'Indicator', 'externalId'])

In [23]:
%time lcia(p, next(qs))

ILCD2011, Ionizing radiation, Comparative Toxic Unit for ecosystems (CTUe) expressing an estimate of the potentially affected fraction of species (PAF) integrated over time and volume per unit mass of a radionucleide emitted (PAF m3 year/kg).
------------------------------------------------------------
  1.86e-05 x 0.00089    = 1.65e-08   antimony-124
  1.51e-05 x 0.0273     = 4.12e-07   antimony-125
  1.04e-05 x 0.639      = 6.65e-06   iodine-131
  4.59e-09 x 133        = 6.13e-07   hydrogen-3
  2.43e-05 x 0.144      = 3.51e-06   cobalt-58
  4.62e-05 x 0.0242     = 1.12e-06   antimony-124
   5.5e-05 x 0.000267   = 1.47e-08   cesium-134
  0.000124 x 0.0275     = 3.41e-06   cesium-134
  2.26e-05 x 0.000968   = 2.19e-08   cobalt-60
  4.78e-06 x 9.97e-10   = 4.77e-15   manganese-54
  4.49e-05 x 0.000355   = 1.59e-08   cesium-137
  0.000172 x 0.0207     = 3.56e-06   carbon-14
  1.37e-05 x 0.0295     = 4.04e-07   silver-110
  6.83e-05 x 4.24e-06   = 2.9e-10    tellurium-123
  5.41e-05 x 0.1

0.0031144941582810586

### Ecoinvent LCIA Computation

In [5]:
e = archive_from_json(os.path.join(catalog_dir, 'ei_lcia.json.gz'))

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz:
**Upstream reference encountered: /data/LCI/Ecoinvent/3.2/undefined

708 new quantity entities added (708 total)
3255 new flow entities added (3255 total)
0 new process entities added (0 total)


In [7]:
from lcatools.providers.ecospold2 import EcospoldV2Archive

In [8]:
ei_32_path = os.path.join(os.path.expanduser('~'),'Dropbox', 'data', 'Ecoinvent', 
                               '3.2')
ei_32_undefined = os.path.join(ei_32_path, 'current_Version_3.2_undefined_ecoSpold02.7z')


In [9]:
E = EcospoldV2Archive(ei_32_undefined, upstream=e, prefix='datasets - public')

Found Extension: 7z


In [10]:
p = E.retrieve_or_fetch_entity(E.list_datasets()[2790])

In [11]:
qe = (x for x in e.quantities() if 'Method' in x.keys())  # or if isLciaMethod

In [12]:
[str(x) for x in p.exchanges()]

[' Input:     [0.15 [kg]] water, decarbonised, at user [By-product classification: allocatable product]',
 ' Input:     [1e-05 [kg]] chlorine, gaseous (CAS 7782-50-5) [By-product classification: allocatable product]',
 ' Input:     [4.92e-13 [unit]] lignite power plant [By-product classification: allocatable product]',
 ' Input:     [0.006 [kg]] water, completely softened, from decarbonised water, at user [By-product classification: allocatable product]',
 ' Input:     [0.00223 [kg]] SOx retained, in lignite flue gas desulfurisation [By-product classification: allocatable product]',
 ' Input:     [0.0893 [kg]] lignite [By-product classification: allocatable product]',
 ' Input:     [0.106 [m3]] Water, cooling, unspecified natural origin (CAS 007732-18-5) [natural resource, in water]',
 'Output:     [0.00267 [m3]] Water (CAS 007732-18-5) [air, unspecified]',
 'Output:     [2e-08 [kg]] Iodine (CAS 007553-56-2) [air, non-urban air or from high stacks]',
 'Output:     [1e-06 [kg]] Methane,

In [18]:
lcia(p, next(qe))

ReCiPe Midpoint (H) w/o LT, terrestrial acidification w/o LT, TAP100 w/o LT
------------------------------------------------------------
      0.56 x 0.000165   = 9.24e-05   Nitrogen oxides
         1 x 0.000198   = 0.000198   Sulfur dioxide
             Total score: 0.0002904 [kg SO2-Eq]


0.0002904

 same system model three different ways:

In [19]:
E_fg = EcospoldV2Archive(os.path.join(ei_32_path, 'current_Version_3.2_apos_ecoSpold02.7z'), 
                         upstream=e, prefix='datasets')

Found Extension: 7z


In [20]:
E_bg = EcospoldV2Archive(os.path.join(ei_32_path, 'current_Version_3.2_apos_lci_ecoSpold02.7z'), 
                         upstream=e, prefix='datasets')

Found Extension: 7z


In [21]:
E_lcia = EcospoldV2Archive(os.path.join(ei_32_path, 'current_Version_3.2_apos_lcia_ecoSpold02.7z'), 
                           upstream=e, prefix='datasets')
for q in e.quantities():
    E_lcia.add(q)

Found Extension: 7z


In [23]:
file = E_fg.list_datasets()[28]

In [24]:
p_fg = E_fg.retrieve_or_fetch_entity(file)

In [25]:
len([x for x in p_fg.exchanges()])

130

In [26]:
p_bg = E_bg.retrieve_or_fetch_entity(file)

In [27]:
len([x for x in p_bg.exchanges()])

1879

In [28]:
f_lcia = E_lcia.retrieve_lcia_scores(file)

In [36]:
q = next(qe)

In [37]:
lcia(p_fg, q)

IMPACT 2002+ (Endpoint), human health, respiratory effects (inorganics)
------------------------------------------------------------
    0.0986 x 0.000105   = 1.04e-05   Particulates, < 2.5 um
   0.00769 x 0.0019     = 1.46e-05   Sulfur dioxide
     0.012 x 1.35e-10   = 1.61e-12   Ammonia
    0.0125 x 0.00212    = 2.67e-05   Nitrogen oxides
  0.000103 x 0.00222    = 2.28e-07   Carbon monoxide, fossil
  0.000103 x 2.55e-06   = 2.63e-10   Carbon monoxide, non-fossil
             Total score: 5.18897e-05 [points]


5.188967521736182e-05

In [38]:
lcia(p_bg, q)
print('aggregated result: %g' %f_lcia.cf(q))

IMPACT 2002+ (Endpoint), human health, respiratory effects (inorganics)
------------------------------------------------------------
   0.00769 x 0.00191    = 1.47e-05   Sulfur dioxide
     0.012 x 7.84e-08   = 9.39e-10   Ammonia
   0.00769 x 9.24e-07   = 7.1e-09    Sulfur dioxide
    0.0125 x 3.38e-06   = 4.24e-08   Nitrogen oxides
    0.0125 x 8.06e-13   = 1.01e-14   Nitrogen oxides
    0.0125 x 1.69e-06   = 2.12e-08   Nitrogen oxides
  0.000103 x 7.26e-07   = 7.47e-11   Carbon monoxide, fossil
  0.000103 x 0.00233    = 2.39e-07   Carbon monoxide, fossil
    0.0986 x 0.000106   = 1.04e-05   Particulates, < 2.5 um
  0.000103 x 2.01e-06   = 2.07e-10   Carbon monoxide, fossil
    0.0986 x 1.38e-07   = 1.37e-08   Particulates, < 2.5 um
     0.012 x 7.09e-09   = 8.49e-11   Ammonia
   0.00615 x 9.68e-13   = 5.96e-15   Sulfur trioxide
   0.00769 x 3.49e-06   = 2.68e-08   Sulfur dioxide
     0.012 x 5.35e-12   = 6.41e-14   Ammonia
  0.000103 x 6.06e-09   = 6.24e-13   Carbon monoxide, non-fos